# Module

In [2]:
import sys
sys.path.append('../')
import os
import torch
import ollama
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)
from src.config import PATH_PROJECT, PROMPT_SYSTEM



cpu


# Test models

In [12]:
import pandas as pd 
df = pd.read_excel(os.path.join(PATH_PROJECT, "data/") + "dataset.xlsx").fillna("Non renseigné")

In [4]:
import requests

def test_ollama():
    try:
        response = requests.get("http://localhost:11434")
        if response.status_code == 200:
            print("Ollama server is UP!")
        else:
            print(f"Ollama server responded with status code: {response.status_code}")
    except requests.exceptions.ConnectionError:
        print("Ollama server is DOWN or unreachable on localhost:11434")

test_ollama()

Ollama server is UP!


In [5]:
from src.inference import get_code, get_code_with_rag

models = ["mistral", "deepseek-coder-v2", "deepseek-r1", "llama3.2"]
models_expert = [ "mistral","deepseek-coder-v2", "deepseek-r1", "llama3.2" ]


In [6]:
from tqdm import tqdm
from src.config import PROMPT_SYSTEM_pandas ,PROMPT_SYSTEM_EXPERT_pandas

#get_code("llama3.2","llama3.2", w_model_expert=True, lan_logic='Datalog')
# get_code("deepseek-r1", "deepseek-r1", w_model_expert=True, prompt_system=PROMPT_SYSTEM_kanren, prompt_system_expert=PROMPT_SYSTEM_EXPERT_kanren,lan_logic='karen')
        
            

In [21]:
get_code("deepseek-coder-v2",model_expert=None, w_model_expert=False, prompt_system=PROMPT_SYSTEM_pandas, prompt_system_expert=PROMPT_SYSTEM_EXPERT_pandas,lan_logic='pandas')
 

 To translate the given diagnostic criteria into Python code using pandas, we need to define specific rules and symptoms associated with each disorder. We will then use these rules to check if a patient's observed symptoms and past conditions meet the criteria for one of the specified disorders. Here's how we can approach this:

### Step-by-Step Reasoning:
1. **Define Criteria for Each Disorder**: 
   - Bipolar I: Must have at least one manic episode, which can be either hypomanic or mixed.
   - Bipolar II: Must have at least one major depressive episode and at least one hypomanic episode.
   - Single Episode Depressive Disorder: Must have at least one major depressive episode.
   - Recurrent Depressive Disorder: Must have multiple major depressive episodes within a certain timeframe, typically more than two episodes in the past year but less than five years.

2. **Extract Relevant Symptoms and Conditions**:
   - From 'Observed_Symptom' and 'Observed_Week', we can check if symptoms are

In [22]:
get_code_with_rag('llama3.2',model_expert='llama3.2', w_model_expert=True, prompt_system=PROMPT_SYSTEM_pandas, prompt_system_expert=PROMPT_SYSTEM_EXPERT_pandas,lan_logic='pandas')
        


Chargement de la base Chroma existante...


c:\Users\FX506\Desktop\CS\3A\MVA\LLM\mva-llm-project\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\FX506\Desktop\CS\3A\MVA\LLM\mva-llm-project\notebooks\..\src\inference.py:267: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=CHROMA_PERSIST_DIR, embedding_function=embedding_model)


Base Chroma prête à être utilisée


In [16]:
df.head()

,PatientID,Disorder,Observed_Symptom,Observed_Week,History_Condition,History_Count,Mood_Episode
0,1,BPD2,depressed_mood,1.5,depressive,1.0,Non renseigné
1,1,BPD2,reduced_concentration,1.2,hypomanic,1.0,Non renseigné
2,1,BPD2,reduced_energy,0.8,Non renseigné,Non renseigné,Non renseigné
3,1,BPD2,increased_talkativeness,0.6,Non renseigné,Non renseigné,Non renseigné
4,2,RDD,depressed_mood,5.7,depressive,1.0,depressive


In [ ]:
import pandas as pd

def diagnose(row):
    # Define relevant symptom and condition names
    symptom_names = ["depressed mood", "manic/hypomanic mood", "elevated/mixed mood", "decreased need for sleep"]
    condition_names = ["Depressive Disorder, single episode", "Depressive Disorder, recurrent (not otherwise specified)", 
                       "Bipolar I Disorder", "Bipolar II Disorder"]

    # Check if there is at least one mood symptom
    has_mood_symptoms = any(sym in row['Observed_Symptom'] for sym in symptom_names)
    
    # Define thresholds for each disorder
    bipolar_i_threshold = 4
    bipolar_ii_threshold = 3
    
    # Determine diagnosis based on presence of mood symptoms and thresholds
    if not has_mood_symptoms:
        return 'No diagnosis'
    elif sum(row['History_Count']) >= bipolar_i_threshold:
        return 'Bipolar I Disorder' if row['Observed_Symptom'].any() else 'Depressive Disorder, recurrent (not otherwise specified)'
    elif any(sym in row['Observed_Symptom'] for sym in ["depressed mood", "manic/hypomanic mood"]) and sum(row['History_Count']) >= bipolar_ii_threshold:
        return 'Bipolar II Disorder'
    elif len([sym for sym, count in zip(row['Observed_Symptom'], row['History_Count']) if any(sym in names for names in condition_names) and count > 1]) == 2:
        return 'Single Episode Depressive Disorder'
    else:
        return 'Recurrent Depressive Disorder'